## I. Збір та анотування даних  
### Опрацюйте один із архівів Wiktionary і витягніть усі синонімні ряди для будь-якої мови. Будь ласка, не обирайте англійську, українську чи російську мови. 
Был выбран шведский язык.  
Пользовался последним дампом - `svwiktionary-latest-pages-articles.xml`  
Скрипт - `parsing.py`  
Результат выполнения скрипта:

In [ ]:
import pandas as pd

In [9]:
unpickled_df = pd.read_pickle("./syns.pkl")
unpickled_df

,word,syn
0,besserwisser['know-it-all'],"[bättrevetare, viktigpetter, messerschmitt]"
1,Sverige['Sweden'],"[landet lagom, moder Svea, Svea rike, Konungar..."
2,barr['barre'],"[balettstång, stång]"
3,samlag['intercourse'],"[sex, knull, älskog]"
4,"knulla['fuck', 'screw', 'have sex']","[banga, borra, bulta, doppa, dra över, dunka, ..."
...,...,...
5493,"pilates boll['exercise ball', 'yoga ball', 'gy...",[balansboll]
5494,växt ätande['herbivorous'],[herbivor]
5495,storhetsvansinne|storhetsvansinn -ig|ig['megal...,"[megaloman, högmodig, övermodig]"
5496,måne|mån kalv['mooncalf'],[druvbörd]


### Визначте рівень згоди між анотувальниками (або inter-annotator agreement) у корпусі NUCLE Error Corpus. 

In [13]:
from itertools import combinations

lines = []
with open("official-2014.combined-withalt.m2") as file:
    lines = file.readlines()

comb = combinations(range(5), 2)
data = {}
#prepare data 
for c in comb:
    data[c] = []


examples = []
curr = []
for line in lines:
    if line!="\n":
        curr.append(line)
    else:
        examples.append(curr)
        curr = [] 

def get_anotator(s):
    return int(s.strip()[-1])

def get_anotation_string(s, sentense_number):
    i = s.find("REQUIRED")
    s = s.strip()[2:i]
    token_range = ""
    count = 0
    for x in s:
        if x=="|":
            break
        count += 1
        token_range += x
    s = s[count+3:]
    count = 0 
    for x in s:
        if x=="|":
            break
        count += 1
    s = s[count+3:]
    correction=""
    count=0
    for x in s:
        if x=="|":
            break
        correction+=x
        count+=1
    return " ".join([str(sentense_number), token_range, correction])


def process_block(l, i):
    anotations = l[1:]
    if len(anotations) > 1:
        anotation_string = []
        t = [""]*5
        curr_anotator = -1
        for x in anotations:
            an = get_anotator(x)
            if an != curr_anotator:
                if curr_anotator!=-1:
                    t[curr_anotator] = anotation_string
                    anotation_string = []
                curr_anotator = an
            anotation_string.append(get_anotation_string(x, i))

        t[curr_anotator] = anotation_string
        #append processed lines to data 
        indexes = [i for i, j in enumerate(t) if j!='']
        if len(indexes)>1:
            for idx in combinations(indexes, 2):
                d1 = t[idx[0]]
                d2 = t[idx[1]]
                g = [d2.count(el) for el in d1]
                c = sum(g)*2/(len(d1)+len(d2))
                data[idx].append(c)


for i, e in enumerate(examples):
    process_block(e, i)


#calculating result 
for k in data:
    print("Score for anotators %s and %s:"%k)
    c = sum(data[k])/len(data[k])
    print(c)

Score for anotators 0 and 1:
0.2304906808555615
Score for anotators 0 and 2:
0.351250171853664
Score for anotators 0 and 3:
0.4226637638858722
Score for anotators 0 and 4:
0.5875675522734346
Score for anotators 1 and 2:
0.40452423868317877
Score for anotators 1 and 3:
0.488079869491824
Score for anotators 1 and 4:
0.48186628186628194
Score for anotators 2 and 3:
0.4665485019978436
Score for anotators 2 and 4:
0.4373219373219373
Score for anotators 3 and 4:
0.46878306878306875
